#Code for CCN 2026 Submission 19: "An Analysis of Neuroidal Memory Formation within D. melanogaster"

##Import FlyWire's Connections Dataset
The default import assumes that the data is housed on a Google user's cloud drive storage. Please edit the file path to point to the correct location of the CSV file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/flywire_connections.csv"
df = pd.read_csv(file_path)

### Versioning:

In [ ]:
import numpy as np
print(np.__version__)


Each row contains:
-----------------------------------------------
* **pre_pt_root_id**: (presynaptic neuron ID)
* **post_pt_root_id**: (postsynaptic neuron ID)
* **neuropil**: (brain region)
* **nt_type**: (Neurotransmitter type) This field labels the predicted neurotransmitter for each synaptic connection (e.g., acetylcholine, GABA, glutamate, etc.)
* **syn_count**: number of synapses between this specific pre-post pair in that neuropil

So in Codex's neuropils view:
--------------------------------
* Each row corresponds to a directed connection between two neurons within one neuropil.
* syn_count: How many synaptic contact points define that edge. Defines strong vs. weak connections.
* Example: A syn_count of 7 means neuron X makes 7 synapses onto neuron Y within the specified neuropil.

In [ ]:
df.size

In [ ]:
df.head()

In [ ]:
df["pre_pt_root_id"].nunique()

In [ ]:
df["post_pt_root_id"].nunique()

## Neuropil Exploration

In [ ]:
# The regions of the brain
# Refers to the brain region where a synapse connects. In flies, brain regions are divided into neuropils—bundled fiber areas like the antennal lobe (AL),
# medulla (ME), lobula (LO), mushroom body (MB), central complex, etc.
# In Codex, each synapse edge reports the neuropil (region) in which it's found. Edges may span different neuropils, and are grouped by region in the graph.
# https://codex.flywire.ai/app/neuropils?dataset=fafb

df["neuropil"].unique()

In [ ]:
df_long = pd.melt(df, id_vars="neuropil", value_vars=["pre_pt_root_id", "post_pt_root_id"],
                  var_name="role", value_name="pt_root_id")

unique_ids_per_neuropil = df_long.groupby("neuropil")["pt_root_id"].nunique().sort_values()
unique_ids_per_neuropil

## Start with **Smallest** brain regions.



1.   LH_L (5723)
2.   OCG (409)



In [ ]:
df_LH_L = df[df["neuropil"]=='LH_L']
df_LH_L.head()

In [ ]:
# In a directed graph adjacency matrix, the row that represents node '720575940608199748' will have 277 nonzero entries.
df_LH_L[df_LH_L['pre_pt_root_id'] == 720575940608199748]

In [ ]:
# Number of edges/connections in LH_L brain region.
df_LH_L.size

In [ ]:
# Number of unique nodes in the LH_L brain region.

# Get unique pre and post node IDs separately
pre_nodes = df_LH_L['pre_pt_root_id'].unique()
post_nodes = df_LH_L['post_pt_root_id'].unique()

# Use set union (mask-based logic)
unique_nodes = set(pre_nodes) | set(post_nodes)

# Count total unique nodes
num_unique_nodes = len(unique_nodes)

print("Total unique nodes:", num_unique_nodes)

In [ ]:
# unique pre nodes in LH_L
df_LH_L["pre_pt_root_id"].nunique()

In [ ]:
# unique post nodes in LH_L
df_LH_L["post_pt_root_id"].nunique()

## Create an adjacency graph of the LH_L subgraph connections.
(binary, unweighted)

*   adj_sparse: CSR sparse adjacency matrix (0/1 entries)
*   node_to_idx: Mapping from neuron ID --> matrix row/col index



In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

def compute_adj_sparse_mat(df_subgraph, directed=False):
  # Get all unique node IDs
  nodes = pd.unique(df_subgraph[['pre_pt_root_id', 'post_pt_root_id']].values.ravel())
  node_to_idx = {node: i for i, node in enumerate(nodes)}

  # Map each ID to its index
  row = df_subgraph['pre_pt_root_id'].map(node_to_idx).values
  col = df_subgraph['post_pt_root_id'].map(node_to_idx).values

  # # Use syn_count as the weight for each edge?
  # data = df_subgraph['syn_count'].values>

  # Use binary weights: 1 for each edge
  data = np.ones_like(row, dtype=int)

  # Build sparse adjacency matrix (directed by default)
  n = len(nodes)
  adj_sparse = coo_matrix((data, (row, col)), shape=(n, n)).tocsr()

  # To make the graph UNDIRECTED:
  if not directed:
    adj_sparse = adj_sparse + adj_sparse.T
    adj_sparse.data[:] = 1  # Ensure it's still binary (not 2s)

  return adj_sparse, node_to_idx

#### Sanity check below 3 cells for pre-->post connections in adj matrix representation.

In [ ]:
# Print the corresponding adj matrix row index to the original df node id.
row_idx = node_to_idx[720575940608199748]
row_idx

In [ ]:
nonzero_cols = adj_sparse[row_idx].nonzero()[1]
print(nonzero_cols.size)
print(nonzero_cols)

In [ ]:
# Used original df to check for an existing connection between a pre node and a post node, as given by nonzero entries of the generated adj matrix.
# MUST map the original id to the zero-index enumerated row of the adj matrix to check against original df.
id_to_find = 720575940608199748
filtered_rows = df_LH_L[df_LH_L['pre_pt_root_id'] == id_to_find]
filtered_rows[filtered_rows["post_pt_root_id"] == 720575940635468991]

Confirmed: Node id 720575940608199748 (corresponding to row index 0 in adj matrix) has 277 outgoing edges. This matches the original df 'pre_pt_root_id' == 720575940635468991 277 entries.

### Compute Clustering Coefficient and Path Length


In [ ]:
# Calculating Clustering Coeff. with directed graph

# Assuming A is a scipy sparse matrix
# Need to use a definition of directed clustering for a directed graph, such as Fagiolo's clustering coefficient:
# For directed graphs, we need to count directed triangles considering edge directions.
import numpy as np
from scipy.sparse import issparse

def calculate_global_clustering_sparse(A, directed=False) -> float:
  if not directed:
    # # MAKE ADJ GRAPH UNDIRECTED
    A = A.maximum(A.T)  # sparse matrix max method

    A2 = A @ A
    A2_sum = A2.sum()
    A2_trace = A2.diagonal().sum()

    # Step 1: Count connected triples = sum(A²) - trace(A²)
    k = A2_sum - A2_trace

    # Step 2: Calculate Global clustering coefficient, only calculate triangles if triples is nonzero:
    if k == 0:
      return 0.0

    # Step 2.1: Triangle count = trace(A³)
    A3 = A2 @ A
    A3_trace = A3.diagonal().sum()
    return A3_trace / k

  else:
    # Directed graph clustering coefficient (Fagiolo 2007) https://arxiv.org/pdf/physics/0612169
    # Step 1: Binarize adjacency matrix so all edges count as 1.
    # Step 2: Symmetrize adjacency A_sym = A + A^T to compute combined in- and out-degrees.
    # Step 3: Compute trace(A_sym^3), counting directed triangles.
    # Step 4: Compute denominator as sum over nodes of di(di - 1) where di = in_degree + out_degree subtracting bilateral edges (A_bin[i, j] == 1 AND A_bin[j, i] == 1)
    # Step 5: Return the ratio as the global clustering coefficient for the directed graph.

    # Step 1: Convert to binary adjacency (edges = 1)
    A_bin = A.copy()
    A_bin.data[:] = 1

    # Step 2: Calculate symmetrized adjacency matrix
    A_sym = A_bin + A_bin.T

    # Step 3: Calculate numerator = trace(A^3)
    A3 = A_sym @ A_sym @ A_sym
    triangles = A3.diagonal().sum()

    # Step 4: Calculate denominator = number of possible triangles
    degrees_tot = np.array(A_sym.sum(axis=1)).flatten()  # in-degree + out-degree
    A2 = A_bin @ A_bin
    num_bilateral_edges = A2.diagonal()
    possible_triangles = 2 * ((degrees_tot * (degrees_tot - 1)) - (2 * num_bilateral_edges))

    total_possible = possible_triangles.sum()
    if total_possible == 0:
      return 0.0

    # Step 5: Global clustering coefficient
    return triangles / total_possible


# Calculating Path Length with undirected graph
# Use built in scipy.sparse.csgraph shortest path algorithms that are optimized for sparse matrices:
from scipy.sparse.csgraph import shortest_path
import numpy as np

def calculate_path_length_sparse(A, directed=False):
  # Ensure adjacency matrix is symmetric if undirected
  if not directed:
    A = A.maximum(A.T)

  # Replace zeros with inf for distances (non-edges)
  # shortest_path treats zeros as no edge by default if you pass 'unweighted=False'
  # so we can just pass the adjacency matrix directly if it encodes weights properly
  # For unweighted graph, set all edges to 1
  A.data[:] = 1

  # Compute all pairs shortest paths
  dist_matrix = shortest_path(A, directed=directed, unweighted=True)

  # Mask out infinite distances (no path)
  finite_distances = dist_matrix[np.isfinite(dist_matrix) & (dist_matrix != 0)]

  # Return average shortest path length (excluding zero distances)
  return np.mean(finite_distances)


def calculate_degree_distribution(A, directed=True):
    if not directed:
        # Make adjacency symmetric
        A = np.maximum(A, A.T)
        # Degree = sum of either rows or columns
        degree = np.sum(A, axis=0)
        return degree
    else:
        # For directed: return both in-degree and out-degree
        in_degree = np.sum(A, axis=0)   # column sums
        out_degree = np.sum(A, axis=1).T  # row sums
        return in_degree, out_degree


In [ ]:
direct = True

# Create adj_sparse
adj_sparse, node_to_idx = compute_adj_sparse_mat(df_LH_L, directed=direct)

print("Shape:", adj_sparse.shape)
print("Edges:", adj_sparse.nnz // 2)  # Undirected, so count each edge once
print("Edge density:", adj_sparse.nnz / (adj_sparse.shape[0] ** 2))

In [ ]:
deg_dist_IN, deg_dist_OUT = calculate_degree_distribution(adj_sparse, direct)
print("In-Degree Distribution: ", deg_dist_IN)
print("Out-Degree Distribution: ", deg_dist_OUT)

In [ ]:
clustCoeff = calculate_global_clustering_sparse(adj_sparse, directed=direct)
print("Clustering Coefficient: ", clustCoeff)

In [ ]:
characteristic_path_length = calculate_path_length_sparse(adj_sparse, directed=direct)
print("Characteristic Path Length:", characteristic_path_length)

#### Find the average degree of the graph (directed / undirected)

In [ ]:
# Assume A is a sparse adjacency matrix (CSR format)
# Directed graph: row sums = out-degree, col sums = in-degree
def compute_graph_average_degree(A, directed=False):
  if directed:
    total_in_deg = A.sum(axis=0).A1.sum()
    total_out_deg = A.sum(axis=1).A1.sum()
    # These should be equal for any directed graph
    assert total_in_deg == total_out_deg

    avg_total_degree_d = (total_in_deg + total_out_deg) / A.shape[0]
    # Check the average total degrees of a node to be within bounds
    # Since the sum of in-degrees equals the sum of out-degrees in a directed graph (each edge contributes one to each), we are effectively computing:
    # avg_total_degree = 2 * total_edges / n
    assert avg_total_degree_d <= 2 * (A.shape[0] - 1)

    return avg_total_degree_d

  else:
    # Undirected
    degrees = A.sum(axis=1)  # or axis=0
    avg_degree = degrees.mean()
    return avg_degree

In [ ]:
# Compute Directed average degree of graph
print("Average Degree:", compute_graph_average_degree(adj_sparse, directed=direct))

### Compute Small-World Threshold

In [ ]:
def compute_small_world_threshold(C_g, L_g, C_rand, L_rand):
  return (C_g / C_rand) / (L_g / L_rand)

In [ ]:
# import C_rand, L_rand values from randGraph.ipynb
C_rand = 0.008435302420417078
L_rand = 2.6529490970869216
sigma = compute_small_world_threshold(clustCoeff, characteristic_path_length, C_rand, L_rand)
print(sigma)

if sigma > 1:
  print("Small world found!")
else:
  print("Not a small world...")

Through the small-world threshold calculation performed above on the LH_L region of the Drosophila brain, we may see that it is a small-world graph. This is defined by a high clustering coefficient and a low characteristic path length, as compared to a random GNP graph with the same number of nodes and average degree as the measured biological subgraph LH_L.

## OCG Brain Region Calculation

In [ ]:
###### OCG #########
df_OCG = df[df["neuropil"]=='OCG']
direct_OCG = True

# Create Sparse Adj Matrix
adj_sparse_OCG, node_to_idx_OCG = compute_adj_sparse_mat(df_OCG, directed=direct_OCG)

deg_dist_OCG_IN, deg_dist_OCG_OUT = calculate_degree_distribution(adj_sparse_OCG, direct_OCG)
print("In-Degree Distribution: ", np.mean(deg_dist_OCG_IN))
print("Out-Degree Distribution: ", np.mean(deg_dist_OCG_OUT))

# Compute Clustering Coefficient
clustCoeff_OCG = calculate_global_clustering_sparse(adj_sparse_OCG, directed=direct_OCG)
print("Clustering Coefficient: ", clustCoeff_OCG)

# Compute Characteristic Path Length
characteristic_path_length_OCG = calculate_path_length_sparse(adj_sparse_OCG, directed=direct_OCG)
print("Path Length: ", characteristic_path_length_OCG)

graph_average_degree_OCG = compute_graph_average_degree(adj_sparse_OCG, directed=direct_OCG)
# Compute Average Degree of Graph
print("Average Degree of Graph:", graph_average_degree_OCG)


In [ ]:
###### OCG #########

# import C_rand, L_rand values from randGraph.ipynb
C_rand_OCG = 0.030600189035916825
L_rand_OCG = 2.6501989548875784
sigma_OCG = compute_small_world_threshold(clustCoeff_OCG, characteristic_path_length_OCG, C_rand_OCG, L_rand_OCG)
print(sigma_OCG)

if sigma_OCG > 1:
  print("Small world found!")
else:
  print("Not a small world...")

#### nt_type Exploration

In [ ]:
# Neurotransmitter type: This field labels the predicted neurotransmitter for each synaptic connection (e.g., acetylcholine, GABA, glutamate, etc.)

df["nt_type"].unique()

#### **Overall connections**: Find the number of times a neuron 'pre_pt_root_id' appears in 'post_pt_root_id'



In [ ]:
# Find the number of times a neuron 'pre_pt_root_id' appears in 'post_pt_root_id'

# Drop duplicates from 'pre_pt_root_id'
unique_col1 = df['pre_pt_root_id'].drop_duplicates()

# Count occurrences in 'post_pt_root_id'
col2_counts = df['post_pt_root_id'].value_counts()

# Map counts to the unique col1 values
result = pd.DataFrame({
    'pre_pt_root_id': unique_col1,
    'count': unique_col1.map(col2_counts).fillna(0).astype(int)
})

result.sort_values(by='count', ascending=False)


# JOIN Algorithm on Brain Region Subgraphs


firing_nodes = a binary vector or list of nodes currently "firing"

For each node j, count how many firing nodes i have A[i, j] = 1

In [ ]:
import numpy as np
import scipy.sparse as sp

# A: scipy.sparse matrix (n x n), A directed binary adjacency matrix.
# firing_nodes: array of indices that are currently firing
# n_nodes: number of nodes in graph
# k_s: firing threshold for the graph
# Returns: np.array of counts of firing inputs per node
# Firing from A directly to C, we should measure and allow recurrence up to 10% of mem_size.
def count_firing_inputs(A, firing_nodes, k_s, n_nodes=None):
  # Recurrence tolerance = 10% of mem_size
  RECURRENCE_TOLERANCE_THRESHOLD = 0.9
  if n_nodes is None:
    n_nodes = A.shape[0]

  # Create a binary firing vector (1 where node is firing)
  firing_vector = np.zeros(n_nodes, dtype=int)
  # Set the mode of these nodes to 'firing'
  firing_vector[firing_nodes] = 1

  # Compute number of firing inputs for each node (incoming edges from firing nodes)
  firing_input_counts = A.T @ firing_vector  # result is dense np.array
  firing_input_counts = np.array(firing_input_counts).flatten()

  # Threshold: fire if count >= k_s
  next_firing_nodes = np.where(firing_input_counts >= k_s)[0]

  # Removed original firing nodes to measure overlap between memA and created memC
  # If the overlap between memA and resultant memC is greater than 10%, then we remove those overlapped nodes from memC, due to recurrence.
  next_firing_nodes_without_overlap = np.setdiff1d(next_firing_nodes, firing_nodes)
  if len(next_firing_nodes) > 0 and len(next_firing_nodes_without_overlap) / len(next_firing_nodes) < RECURRENCE_TOLERANCE_THRESHOLD:
    next_firing_nodes = next_firing_nodes_without_overlap

  # Number of firing nodes that meet the threshold for child memory creation:
  num_meeting_threshold = len(next_firing_nodes)

  return next_firing_nodes, num_meeting_threshold


# k_s = median value of subgraph.syn_count
# Synapse Count in a connection represents the number of synapses in a connectio; otherwise described as
# axon-->dendrite connections.
# Here, we use the synapse count to average over the number of connections to find a median connection strength.
def compute_k_s_threshold(df_subgraph) -> int:
  return df_subgraph['syn_count'].median()

import random
# Set Global Seed
random.seed(0)
# node_to_idx_dict: Dictionary that maps pre, post node id's --> adj sparse matrix indices.
# mem_size: desired memory size
def generate_random_indices_for_memory(node_to_idx_dict, mem_size):
  upper_range_of_indices = len(node_to_idx_dict)
  memory_indices = random.sample(range(0, upper_range_of_indices), mem_size)
  return memory_indices


#### Compute the k_s threshold for a brain region, and simulate firing a random subset of nodes to represent Memory A. The connected nodes will also fire into Memory C if they meet the k_s threshold.


In [ ]:
# Compute the synapse strength to find the k_s, firing threshold
k_s_LH_L = compute_k_s_threshold(df_LH_L)
print(k_s_LH_L)


# Memory A
# Grab a random subset of nodes through the node_to_idx dictionary, as represented by their indices.
memory_A_size = 100
memory_A = generate_random_indices_for_memory(node_to_idx, memory_A_size)

# Fire Memory A, and observe the number of firing nodes that create memory C. Also included: the indices of memory C.
activated_nodes, memory_C_size = count_firing_inputs(adj_sparse, memory_A, k_s_LH_L)
print("Size of Memory C: ", memory_C_size)
print("Memory C: ", activated_nodes)

### Plot varying sizes of Memory A to fire and create Memory C in a brain region.

In [ ]:
# Try varying sizes of memory A
import numpy as np
import matplotlib.pyplot as plt


# Plots memA against fired memC over a range of sizes for memA
def plot_memA_firing_against_memC_creation_single_trials(memory_A_sizes, memory_C_sizes, node_to_idx_dict, adj_matrix, k_s, memory_A_size_MIN, memory_A_size_MAX, brain_region, increment=10, num_trials=20):
  for size in range(memory_A_size_MIN, memory_A_size_MAX, increment):
    memory_A_sizes.append(size)
    # Random subset of nodes for Memory A
    memory_A = generate_random_indices_for_memory(node_to_idx_dict, size)

    # Fire memory A
    activated_nodes, memory_C_size = count_firing_inputs(adj_matrix, memory_A, k_s)

    memory_C_sizes.append(memory_C_size)
    print(f"Memory A size: {size}, Memory C size: {memory_C_size}")

# Plots several rounds of trials of: memA against fired memC over a range of sizes for memA
def plot_memA_firing_against_memC_creation(memory_A_sizes, memory_C_sizes, node_to_idx_dict, adj_matrix, k_s, memory_A_size_MIN, memory_A_size_MAX, brain_region, graphType, increment=10, num_trials=20):
  num_points = ((memory_A_size_MAX - memory_A_size_MIN) // increment) + 1
  memory_C_sizes = [0] * num_points
  memory_A_sizes = list(range(memory_A_size_MIN, memory_A_size_MAX + 1, increment))
  for i in range(num_trials):
    for idx, size in enumerate(memory_A_sizes):
      # Random subset of nodes for Memory A
      memory_A = generate_random_indices_for_memory(node_to_idx_dict, size)

      # Fire memory A
      activated_nodes, memory_C_size = count_firing_inputs(adj_matrix, memory_A, k_s)

      memory_C_sizes[idx] += memory_C_size
      print(f"Memory A size: {size}, Memory C size: {memory_C_size}, size:{size}, index:{size//increment}, num_points: {num_points}")

  # Average out mem_C size values over trials
  memory_C_sizes = [x / num_trials for x in memory_C_sizes]

  # Plot the relationship
  plt.figure(figsize=(6,4))
  plt.plot(memory_A_sizes, memory_C_sizes, marker='o', color='black', linestyle='-', markerfacecolor='gray')
  plt.xlabel("Memory A Size")
  plt.ylabel("Memory C Size", rotation=0, labelpad=40)
  plt.title(f"Memory A Size vs Memory C Size of Brain Region {brain_region}")
  plt.grid(False)
  plt.savefig(f"stability{graphType}.pdf", format='pdf', bbox_inches='tight')
  plt.show()


# Plots several rounds of trials of: memA against fired memC over a range of sizes for memA, with standard deviation error bars
def plot_memA_firing_against_memC_creation_with_error_bars(memory_A_sizes, memory_C_sizes, node_to_idx_dict, adj_matrix, k_s, memory_A_size_MIN, memory_A_size_MAX, brain_region, increment=10, num_trials=20):
  # Setup: discrete bins of memory A
  memory_A_sizes = list(range(memory_A_size_MIN, memory_A_size_MAX + 1, increment))
  num_points = len(memory_A_sizes)

  # Collect trial results (list of lists)
  all_trials = [[] for _ in range(num_points)]

  # Run trials num_trials times
  for i in range(num_trials):
    for idx, size in enumerate(memory_A_sizes):
      # Random subset of nodes for Memory A
      memory_A = generate_random_indices_for_memory(node_to_idx_dict, size)
      # Fire memory A
      activated_nodes, memory_C_size = count_firing_inputs(adj_matrix, memory_A, k_s)
      # add memory_C size to the respective list
      all_trials[idx].append(memory_C_size)

  # Compute mean and standard deviation across trials
  memory_C_sizes = [np.mean(vals) for vals in all_trials]
  memory_C_stds  = [np.std(vals) for vals in all_trials]  # or use np.std(vals)/np.sqrt(num_trials) for standard error

  # Plot the mean line for error regions
  plt.plot(
      memory_A_sizes,
      memory_C_sizes,
      '-o',
      color='black',
      markerfacecolor='gray',
      label='Mean ± Std'
  )

  # Add the shaded error region
  plt.fill_between(
      memory_A_sizes,
      np.array(memory_C_sizes) - np.array(memory_C_stds),
      np.array(memory_C_sizes) + np.array(memory_C_stds),
      color='coral',
      alpha=0.3  # transparency level
  )

  plt.xlabel("Memory A Size")
  plt.ylabel("Memory C Size", rotation=0, labelpad=40)
  plt.title(f"Memory A Size vs Memory C Size of Brain Region {brain_region}")
  plt.grid(False)
  plt.legend()
  plt.savefig("stabilityOCG.pdf", format='pdf', bbox_inches='tight')
  plt.show()


### LH_L: Plot varying sizes of Memory A to fire and create Memory C in a brain region.

In [ ]:
# Compute the synapse strength to find the k_s, firing threshold
k_s_LH_L = compute_k_s_threshold(df_LH_L)
print(k_s_LH_L)

# Generate memory A and fire to get memory C, then plot.
brain_region_name = "LH_L"
memory_A_size_MIN = 1
memory_A_size_MAX = 1000  # different A sizes to test
k_s = k_s_LH_L
memory_A_sizes = []  # for plotting
memory_C_sizes = []  # for plotting
node_to_idx_dict = node_to_idx
adj_matrix = adj_sparse
increment = 20
num_trials = 1

# Plot varying sizes of fired nodes in Memory A against firing nodes of created Memory C
plot_memA_firing_against_memC_creation(memory_A_sizes, memory_C_sizes, node_to_idx_dict, adj_matrix, k_s,
                                       memory_A_size_MIN, memory_A_size_MAX, brain_region_name, "LHL", increment, num_trials)


### OCG: Plot varying sizes of Memory A to fire and create Memory C in a brain region.

In [ ]:
# Compute the synapse strength to find the k_s, firing threshold
k_OCG = compute_k_s_threshold(df_OCG)
print(k_OCG)

# Generate memory A and fire to get memory C, then plot.
brain_region_name_OCG = "OCG"
memory_A_size_MIN_OCG = 100
memory_A_size_MAX_OCG = 125  # different A sizes to test
k_s_OCG = k_OCG
memory_A_sizes_OCG = []  # for plotting
memory_C_sizes_OCG = []  # for plotting
node_to_idx_dict_OCG = node_to_idx_OCG
adj_matrix_OCG = adj_sparse_OCG
increment_OCG = 1
num_trials_OCG = 30

# Plot varying sizes of fired nodes in Memory A against firing nodes of created Memory C
plot_memA_firing_against_memC_creation(memory_A_sizes_OCG, memory_C_sizes_OCG, node_to_idx_dict_OCG, adj_matrix_OCG, k_s_OCG, memory_A_size_MIN_OCG,
                                       memory_A_size_MAX_OCG, brain_region_name_OCG, "OCG", increment_OCG, num_trials_OCG)


In [ ]:
plot_memA_firing_against_memC_creation_with_error_bars(memory_A_sizes_OCG, memory_C_sizes_OCG, node_to_idx_dict_OCG, adj_matrix_OCG, k_s_OCG, memory_A_size_MIN_OCG,
                                       memory_A_size_MAX_OCG, brain_region_name_OCG, increment_OCG, num_trials_OCG)

## Stability

### Fire Memory A and Memory B together to create C in a single brain region (Shared Representation).


For the shared representation, **the size of union(A,B) should be of size 2r - (r^2 / n)**, where the resulting memory C should be about **r**.

*   **(r^2 / n)** = intersection of A and B
*   **r** = num of nodes of mem A, B, C
*   **n** = total num of nodes in graph
*   As A and B need to be fired at the same time, we can simply take a single, combined set of size 2r - (r^2 / n) and fire that in a 1-step JOIN to get memory C.

1.   Memory A: random subset of nodes in a brain region, overlap with B is ok.
2.   Memory B: random subset of nodes in a brain region, overlap with A is ok.
3.   Memroy C: resulting subset of nodes in a brain region that meet the firing threshold, k.

If the Interference Tolerance is breached, take only unique nodes for memory C.



In [ ]:
# Plots several rounds of trials of: memA against fired memC over a range of sizes for memA, with error regions
def plot_memAB_firing_against_memC_creation_with_error_bars(memory_AB_sizes, memory_C_sizes, node_to_idx_dict, adj_matrix, k_s, memory_AB_size_MIN, memory_AB_size_MAX, brain_region, increment=10, num_trials=20):
  # Setup: discrete bins of memory A
  memory_AB_sizes = list(range(memory_AB_size_MIN, memory_AB_size_MAX + 1, increment))
  num_points = len(memory_AB_sizes)

  # Collect trial results (list of lists)
  all_trials = [[] for _ in range(num_points)]

  # Run trials num_trials times
  for i in range(num_trials):
    for idx, size in enumerate(memory_AB_sizes):
      # Random subset of nodes for Memory AB
      memory_AB = generate_random_indices_for_memory(node_to_idx_dict, size)
      # Fire memory AB
      activated_nodes, memory_C_size = count_firing_inputs(adj_matrix, memory_AB, k_s)
      # add memory_C size to the respective list
      all_trials[idx].append(memory_C_size)

  # Compute Mean and Standard Deveiation across all trials
  memory_C_sizes = [np.mean(vals) for vals in all_trials]
  memory_C_stds  = [np.std(vals) for vals in all_trials]  # or use np.std(vals)/np.sqrt(num_trials) for standard error

  # Plot the mean line
  plt.plot(
      memory_AB_sizes,
      memory_C_sizes,
      '-o',
      color='black',
      markerfacecolor='gray',
      label='Mean ± Std'
  )

  # Add the shaded error region
  plt.fill_between(
      memory_AB_sizes,
      np.array(memory_C_sizes) - np.array(memory_C_stds),
      np.array(memory_C_sizes) + np.array(memory_C_stds),
      color='tomato',
      alpha=0.3  # transparency level
  )

  plt.xlabel("Union of Memory A and B Size")
  plt.ylabel("Memory C Size", rotation=0, labelpad=40)
  plt.title(f"Union of Memory A and Memory B Size vs. Memory C Size of Brain Region {brain_region}")
  plt.grid(False)
  plt.legend()
  plt.show()


In [ ]:
# Plots several rounds of trials of: memA against fired memC over a range of sizes for memA, with standard deviation error bars
def plot_memAB_firing_against_memC_creation_with_error_bars_and_UNION_sizes(r_min, r_max, n, memory_AB_sizes, memory_C_sizes, node_to_idx_dict, adj_matrix, k_s, memory_AB_size_MIN, memory_AB_size_MAX, brain_region, graphType, increment=10, num_trials=20):
  # Setup: discrete bins of memory A
  memory_AB_sizes = list(range(memory_AB_size_MIN, memory_AB_size_MAX + 1, increment))
  num_points = len(memory_AB_sizes)

  # Collect trial results (list of lists)
  all_trials = [[] for _ in range(num_points)]

  # Run trials num_trials times
  for i in range(num_trials):
    for idx, size in enumerate(memory_AB_sizes):
      # Random subset of nodes for Memory AB
      memory_AB = generate_random_indices_for_memory(node_to_idx_dict, size)
      # Fire memory AB
      activated_nodes, memory_C_size = count_firing_inputs(adj_matrix, memory_AB, k_s)
      # add memory_C size to the respective list
      all_trials[idx].append(memory_C_size)

  # Compute Mean and Standard Deveiation across all trials
  memory_C_sizes = [np.mean(vals) for vals in all_trials]
  memory_C_stds  = [np.std(vals) for vals in all_trials]  # or use np.std(vals)/np.sqrt(num_trials) for standard error

  # Plot the mean line
  plt.plot(
      memory_AB_sizes,
      memory_C_sizes,
      '-o',
      color='black',
      markerfacecolor='gray',
      label='Mean ± Std'
  )

  # Add the shaded error region
  plt.fill_between(
      memory_AB_sizes,
      np.array(memory_C_sizes) - np.array(memory_C_stds),
      np.array(memory_C_sizes) + np.array(memory_C_stds),
      color='tomato',
      alpha=0.3  # transparency level
  )

  # ---- Add UNION sizes y = 2r - (r^2 / n) for same r range ----
  r_sizes = list(range(r_min, r_max + 1, increment))
  UNION_sizes = [2 * r_size - (r_size**2) // n for r_size in r_sizes]

  plt.plot(  # flip r_sizes and Union sizes so that they overlay on graph as (Union sizes, r_size created)
      UNION_sizes,
      r_sizes,
      '--',
      label=r"UNION sizes vs. r sizes $y = 2r - \frac{r^{2}}{n}$"
  )

  plt.xlabel("Union of Memory A and B Size")
  plt.ylabel("Memory C Size", rotation=0, labelpad=40)
  plt.title(f"Union of Memory A and Memory B Size vs. Memory C Size of Brain Region {brain_region}")
  plt.grid(False)
  plt.legend(loc='upper right')
  plt.savefig(f"stabilityUNIONOCG{graphType}.pdf", format='pdf', bbox_inches='tight')
  plt.show()

In [ ]:
# Jointly fire Mem A and Mem B together to create Mem C. Use a one-step method by approximating the size union(A,B).
def JOIN_Stability_Testing(df_subgraph, r_min, r_max, n, brain_region_name, node_to_idx_dict, adj_matrix, graphType):
  # Compute the size of Union(A,B) for r_min and r_max, respectively. 2 different sizes are calculated for a range of firing values.
  union_AB_min_size = 2 * r_min - ((r_min ** 2) // n)
  union_AB_max_size = 2 * r_max - ((r_max ** 2) // n)

  # Compute the synapse strength to find the k_s, firing threshold
  k_s_subgraph = compute_k_s_threshold(df_subgraph)
  print(k_s_subgraph)

  # Generate memory A and fire to get memory C, then plot.
  k_s = k_s_subgraph

  memory_AB_size_MIN = union_AB_min_size  # different Union(A,B) sizes to test
  memory_AB_size_MAX = union_AB_max_size  # different Union(A,B) sizes to test
  memory_AB_sizes = []  # for plotting
  memory_C_sizes = []  # for plotting

  increment = 1
  num_trials = 30

  # Plot varying sizes of fired nodes in Memory A against firing nodes of created Memory C
  plot_memAB_firing_against_memC_creation_with_error_bars_and_UNION_sizes(r_min, r_max, n, memory_AB_sizes, memory_C_sizes, node_to_idx_dict, adj_matrix, k_s, memory_AB_size_MIN,
                                        memory_AB_size_MAX, brain_region_name, graphType, increment, num_trials)


# params specifically needed for 2-memory firing.
r_min_per_mem_OCG = 100
r_max_per_mem_OCG = 140
n_OCG = 409
# other params needed
df_OCG = df[df["neuropil"]=='OCG']
brain_region_name_OCG = "OCG"
node_to_idx_dict_OCG = node_to_idx_OCG
adj_matrix_OCG = adj_sparse_OCG

# Fire memory A and memory B together, with varying sizes of 100-125 nodes each.
JOIN_Stability_Testing(df_OCG, r_min_per_mem_OCG, r_max_per_mem_OCG, n_OCG, brain_region_name_OCG, node_to_idx_dict_OCG, adj_matrix_OCG, "fullUnionRange")

It looks like the best UNION range is between [200, 220] where the anticipated sizes fall in between a standard deviation of the error region.


In [ ]:
# params specifically needed for 2-memory firing.
r_min_per_mem_OCG = 115
r_max_per_mem_OCG = 132
n_OCG = 409
# other params needed
df_OCG = df[df["neuropil"]=='OCG']
brain_region_name_OCG = "OCG"
node_to_idx_dict_OCG = node_to_idx_OCG
adj_matrix_OCG = adj_sparse_OCG

# Fire memory A and memory B together, with varying sizes of 100-125 nodes each.
JOIN_Stability_Testing(df_OCG, r_min_per_mem_OCG, r_max_per_mem_OCG, n_OCG, brain_region_name_OCG, node_to_idx_dict_OCG, adj_matrix_OCG, "partialUnionRange")

## Capacity

Capacity Testing

Run Join again and again creating Memory C1, C2, C3, ... [D]
Every time we run and create Memory C, run a check to see how much interference there is between resulting Memory D's.

Interference is what we are trying to find out! (What % of nodes are interfering with others, how many memories can we create using new A and B each generation?)

*A node can belong to many memories.*

**Local Interference Rate:** Check every individual node in C and in all other memories D and if more than 50% of the nodes in D are shared with C, then those 2 memories are considered to be interfering.
* (# of interfering memories) / (# of all memories) > 10% of preexisting memories
* Interference --> can not form the new memory, designate the graph to be 'at capacity'.

**Global Interference:** If **more than 10% of all nodes in the whole graph** (OCG) belong to 2+ memories --> Interference Found!

In [ ]:
# Jointly fire Mem A and Mem B together to create Mem C. Use a one-step method by approximating the size union(A,B).
def JOIN_Capacity_Simulation(df_subgraph, r_min, r_max, n, brain_region_name, node_to_idx_dict, adj_matrix):
  # Compute the size of Union(A,B) for r_min and r_max, respectively. 2 different sizes are calculated for a range of firing values.
  union_AB_min_size = 2 * r_min - ((r_min ** 2) // n) - 1
  union_AB_max_size = 2 * r_max - ((r_max ** 2) // n) - 1

  # Compute the synapse strength to find the k_s, firing threshold
  k_s_subgraph = compute_k_s_threshold(df_subgraph)
  print(k_s_subgraph)

  # Generate memory A and fire to get memory C, then plot.
  k_s = k_s_subgraph

  memory_AB_size_MIN = union_AB_min_size  # different Union(A,B) sizes to test
  memory_AB_size_MAX = union_AB_max_size  # different Union(A,B) sizes to test
  memory_AB_sizes = []  # for plotting
  stable_memories_sizes = []  # for plotting

  increment = 1

  # Plot varying sizes of fired nodes in Memory A against firing nodes of created Memory C
  memory_AB_sizes, stable_memories_sizes, stable_memories_stds = plot_memAB_firing_against_memC_Capacity_Testing_with_error_bars(memory_AB_sizes, stable_memories_sizes, node_to_idx_dict, adj_matrix, k_s, memory_AB_size_MIN,
                                        memory_AB_size_MAX, brain_region_name, "OCG", increment)
  return memory_AB_sizes, stable_memories_sizes, stable_memories_stds


# Plots several rounds of trials of: memA against fired memC over a range of sizes for memA, with standard deviation error bars
def plot_memAB_firing_against_memC_Capacity_Testing_with_error_bars(memory_AB_sizes, stable_memories_sizes, node_to_idx_dict, adj_matrix, k_s, memory_AB_size_MIN, memory_AB_size_MAX, brain_region, graphType, increment=1, num_trials=30):
  # Setup Local and Global Interference expectations
  LOCAL_INTERFERENCE_THRESHOLD = 0.5
  GLOBAL_INTERFERENCE_THRESHOLD = 0.1

  # Setup: discrete bins of memory AB
  memory_AB_sizes = list(range(memory_AB_size_MIN, memory_AB_size_MAX + 1, increment))

  # Store num stable memories per Union(A,B) size, across num_trials
  stable_memories_per_AB_size = [[] for _ in range(len(memory_AB_sizes))]
  Global_interference_per_AB_size = [[] for _ in range(len(memory_AB_sizes))]

  for i in range(num_trials):
    # Collect Memory results (list of lists) and Global Interference results
    num_points = len(memory_AB_sizes)
    all_trials = [[] for _ in range(num_points)]
    Global_Interference_Levels_per_AB_Size = [0.0] * num_points

    # Perform Capacity Testing 1x for each size in range of AB_Union_min, AB_Union_max
    for idx, size in enumerate(memory_AB_sizes):
      num_interfering_memories = 0
      Global_Interference_Level = 0.0
      interference_capacity_reached = False
      # Generate Memories and Evaluate Interference
      while not interference_capacity_reached:
        # Generate Memories
        # Random subset of nodes for Memory AB
        memory_AB = generate_random_indices_for_memory(node_to_idx_dict, size)
        # Fire memory AB
        activated_nodes, memory_C_size = count_firing_inputs(adj_matrix, memory_AB, k_s)

        # Evaluate Interference between memory C and all other memories D
        num_all_memories_D = len(all_trials[idx])
        for mem in all_trials[idx]:
          recurrence_nodes = np.intersect1d(activated_nodes, mem)
          # Local interference found
          if len(recurrence_nodes) / len(activated_nodes) >= LOCAL_INTERFERENCE_THRESHOLD:
            num_interfering_memories += 2

        # Check for global interference threshold (+1 is to count the last memory C generated in addition to the Memories D)
        Global_Interference_Level = num_interfering_memories / (num_all_memories_D + 1)
        if Global_Interference_Level >= GLOBAL_INTERFERENCE_THRESHOLD:
          interference_capacity_reached = True
          Global_Interference_Levels_per_AB_Size[idx] = Global_Interference_Level
        else:
          all_trials[idx].append(activated_nodes)

      # Store this trial. Keep track of the number of memories for this Union(A,B) size AND the Global interference
      stable_memories_per_AB_size[idx].append(len(all_trials[idx]))
      Global_interference_per_AB_size[idx].append(Global_Interference_Levels_per_AB_Size[idx])


  # Compute Mean and Standard Deviation of num stable memories, across all trials
  stable_memories_sizes = [np.mean(vals) for vals in stable_memories_per_AB_size]
  stable_memories_stds  = [np.std(vals) for vals in stable_memories_per_AB_size]  # or use np.std(vals)/np.sqrt(num_trials) for standard error

  # Print Results from trials
  zipped_trial_data = list(zip(memory_AB_sizes, Global_interference_per_AB_size, stable_memories_sizes))
  print("AB_size, GLOBAL INTERFERENCE LEVEL, Number of Memories D:")
  for ab_size, global_level, memories in zipped_trial_data:
    print(ab_size, np.mean(global_level), memories)

  # Plot the mean line for error region
  plt.plot(
      memory_AB_sizes,
      stable_memories_sizes,
      '-o',
      color='black',
      markerfacecolor='gray',
      label='Mean ± Std'
  )

  # Add the shaded error region
  plt.fill_between(
      memory_AB_sizes,
      np.array(stable_memories_sizes) - np.array(stable_memories_stds),
      np.array(stable_memories_sizes) + np.array(stable_memories_stds),
      color='tomato',
      alpha=0.3  # transparency of the shaded area
  )

  plt.xlabel("Union of Memory A and B Size")
  plt.ylabel("Number of Memories\n at Capacity", rotation=0, labelpad=50)
  plt.title(f"Union of Memory A and Memory B Size vs. The Number\n of Memories at Capacity of {brain_region}")
  plt.grid(False)
  plt.legend()
  plt.savefig(f"capacityChart_{graphType}.pdf", format='pdf', bbox_inches='tight')
  plt.show()
  return memory_AB_sizes, stable_memories_sizes, stable_memories_stds


In [ ]:
# CAP Test for JOIN on OCG

# params specifically needed for 2-memory firing.
r_min_per_mem_OCG = 100
r_max_per_mem_OCG = 125
n_OCG = 409
# other params needed
df_OCG = df[df["neuropil"]=='OCG']
brain_region_name_OCG = "Brain Region OCG"
node_to_idx_dict_OCG = node_to_idx_OCG
adj_matrix_OCG = adj_sparse_OCG

# Fire memory A and memory B together, with varying sizes of 100-125 nodes each.
OCG_memory_AB_sizes, OCG_stable_memories_sizes, OCG_stable_memories_stds = JOIN_Capacity_Simulation(df_OCG, r_min_per_mem_OCG, r_max_per_mem_OCG, n_OCG, brain_region_name_OCG, node_to_idx_dict_OCG, adj_matrix_OCG)

### Capacity Testing on Watts-Strogatz graphs.

In [ ]:
import numpy as np
import itertools
from scipy.sparse import coo_matrix

# Jointly fire Mem A and Mem B together to create Mem C. Use a one-step method by approximating the size union(A,B).
def JOIN_WS_Capacity_Simulation(k_s_subgraph, r_min, r_max, n, brain_region_name, node_to_idx_dict, adj_matrix):
  # Compute the size of Union(A,B) for r_min and r_max, respectively. 2 different sizes are calculated for a range of firing values.
  union_AB_min_size = 2 * r_min - ((r_min ** 2) // n) - 1
  union_AB_max_size = 2 * r_max - ((r_max ** 2) // n) - 1

  # Generate memory A and fire to get memory C, then plot.
  k_s = k_s_subgraph

  memory_AB_size_MIN = union_AB_min_size  # different Union(A,B) sizes to test
  memory_AB_size_MAX = union_AB_max_size  # different Union(A,B) sizes to test
  memory_AB_sizes = []  # for plotting
  memory_C_sizes = []  # for plotting

  increment = 1
  num_trials = 30

  # Plot varying sizes of fired nodes in Memory A against firing nodes of created Memory C
  memory_AB_sizes, stable_memories_sizes, stable_memories_stds = plot_memAB_firing_against_memC_Capacity_Testing_with_error_bars(memory_AB_sizes, memory_C_sizes, node_to_idx_dict, adj_matrix, k_s, memory_AB_size_MIN,
                                        memory_AB_size_MAX, brain_region_name, "WS", increment, num_trials)
  return memory_AB_sizes, stable_memories_sizes, stable_memories_stds


# Generate a Watts–Strogatz small-world network adjacency matrix in sparse format.
# n = Number of nodes
# p = Rewiring probability
# k = Each node is connected to k nearest neighbors in a ring lattice (graph degree/ for directed graph)
# directed = Whether the graph is directed (True) or undirected (False)
# seed = Random seed for reproducibility
def create_ws_graph_sparse(n: int, p: float, knn: int = -1, directed: bool = True, seed=None):
    if knn == -1:
        knn = int((n + np.log(n)) // 2)
    if knn > n:
        raise ValueError("knn cannot be larger than n")
    if knn == n:
        return coo_matrix(np.eye(n)).tocsr()

    rng = np.random.default_rng(seed)

    # Collect edges for sparse adj matrix
    rows = []
    cols = []

    pool = rng.random(n * knn)
    for pix, (n_ix, iy) in enumerate(itertools.product(range(n), range(1, knn + 1))):
        idx = n_iy = (n_ix + iy) % n  # lattice neighbor
        if pool[pix] < p:
            # rewire each edge of k-nearest neighbors with probability p to a random node
            while idx == n_ix or idx == n_iy:
                idx = rng.integers(0, n)
        rows.append(n_ix)
        cols.append(idx)

        if not directed:
            rows.append(idx)
            cols.append(n_ix)

    data = np.ones(len(rows), dtype=int)
    adj_sparse = coo_matrix((data, (rows, cols)), shape=(n, n)).tocsr()

    # Make sure it stays binary (no multi-edges summing to >1)
    adj_sparse.data[:] = 1

    return adj_sparse

# direct = Directed Boolean
# n = Number of nodes
# d = degree of biological graph (drosophila)
# p = Edge probability
def compute_p(n, d, directed):
  if directed:
    return d / (2 * (n - 1))
  else:
    return d / (n - 1)

#### Try a range of n nodes to observe the change in interference and stable memories.



*   n = 409
*   d = 12

Emulates the Stability of OCG over different AB Union sizes well.




In [ ]:
import math

# Try various n node sizes over a range of [380, 420] to understand the effect on
varying_n_over_WS = [409]
for n in varying_n_over_WS:
  # Compute Adj_Matrix, Small-World Metrics for WS Graph using OCG Small-World Metrics
  ws_OCG_n = n
  ws_OCG_d = 12
  ws_OCG_k = int(k_s_OCG)
  print("k: ", ws_OCG_k)
  ws_OCG_directed = True
  ws_OCG_p = compute_p(ws_OCG_n, ws_OCG_d, ws_OCG_directed)
  print("p: ", ws_OCG_p)
  ws_OCG_knn = math.ceil(ws_OCG_d / 2)
  ws_adj_matrix_OCG = create_ws_graph_sparse(ws_OCG_n, ws_OCG_p, ws_OCG_knn)

  # Compute WS Small World Metrics
  deg_dist_WS_OCG_IN, deg_dist_WS_OCG_OUT = calculate_degree_distribution(ws_adj_matrix_OCG, ws_OCG_directed)
  print("In-Degree Distribution: ", np.mean(deg_dist_WS_OCG_IN))
  print("Out-Degree Distribution: ", np.mean(deg_dist_WS_OCG_OUT))

  # Compute Clustering Coefficient
  clustCoeff_WS_OCG = calculate_global_clustering_sparse(ws_adj_matrix_OCG, ws_OCG_directed)
  print("Clustering Coefficient: ", clustCoeff_WS_OCG)

  # Compute Characteristic Path Length
  characteristic_path_length_WS_OCG = calculate_path_length_sparse(ws_adj_matrix_OCG, ws_OCG_directed)
  print("Path Length: ", characteristic_path_length_WS_OCG)

  # Compute Average Degree of Graph
  print("Average Degree of Graph:", compute_graph_average_degree(ws_adj_matrix_OCG, ws_OCG_directed))

  # Fire Memory A and Memory B in the WS Small Worlds Graph to simulate JOIN
  r_min_per_mem_OCG = 100
  r_max_per_mem_OCG = 125

  # other params needed
  ws_brain_region_name_OCG = "WS Graph of OCG Qualities (n, d, k)"
  node_to_idx_dict_OCG = [0] * ws_OCG_n

  # Fire memory A and memory B together, with varying sizes of 100-125 nodes each.
  WS_memory_AB_sizes, WS_stable_memories_sizes, WS_stable_memories_stds = JOIN_WS_Capacity_Simulation(ws_OCG_k, r_min_per_mem_OCG, r_max_per_mem_OCG, ws_OCG_n, ws_brain_region_name_OCG, node_to_idx_dict_OCG, ws_adj_matrix_OCG)

### Plot OCG and WS Capacity Distributions on Top of One Another

In [ ]:
def JOIN_BOTH_Capacity_Simulation(df_OCG, r_min_per_mem_OCG, r_max_per_mem_OCG, n_OCG, brain_region_name_OCG, node_to_idx_dict_OCG, adj_matrix_OCG,
    ws_OCG_k, ws_OCG_n, ws_brain_region_name_OCG, ws_adj_matrix_OCG):
  OCG_memory_AB_sizes, OCG_stable_memories_sizes, OCG_stable_memories_stds = JOIN_Capacity_Simulation(df_OCG, r_min_per_mem_OCG, r_max_per_mem_OCG, n_OCG, brain_region_name_OCG, node_to_idx_dict_OCG, adj_matrix_OCG)
  WS_memory_AB_sizes, WS_stable_memories_sizes, WS_stable_memories_stds = JOIN_WS_Capacity_Simulation(ws_OCG_k, r_min_per_mem_OCG, r_max_per_mem_OCG, ws_OCG_n, ws_brain_region_name_OCG, node_to_idx_dict_OCG, ws_adj_matrix_OCG)

  ###### PLOT OCG ######
  # Plot the mean line for error region
  plt.plot(
      OCG_memory_AB_sizes,
      OCG_stable_memories_sizes,
      '-o',
      color='black',
      markerfacecolor='red',
      label='OCG Mean ± Std'
  )

  # Add the shaded error region
  plt.fill_between(
      OCG_memory_AB_sizes,
      np.array(OCG_stable_memories_sizes) - np.array(OCG_stable_memories_stds),
      np.array(OCG_stable_memories_sizes) + np.array(OCG_stable_memories_stds),
      color='tomato',
      alpha=0.3  # transparency of the shaded area
  )

  ###### PLOT WS ######
  # Plot the mean line for error region
  plt.plot(
      WS_memory_AB_sizes,
      WS_stable_memories_sizes,
      '-o',
      color='black',
      markerfacecolor='blue',
      label='WS Mean ± Std'
  )

  # Add the shaded error region
  plt.fill_between(
      WS_memory_AB_sizes,
      np.array(WS_stable_memories_sizes) - np.array(WS_stable_memories_stds),
      np.array(WS_stable_memories_sizes) + np.array(WS_stable_memories_stds),
      color='blue',
      alpha=0.3  # transparency of the shaded area
  )

  plt.xlabel("Union of Memory A and B Size")
  plt.ylabel("Number of Memories\n at Capacity", rotation=0, labelpad=60)
  plt.title(f"Union of Memory A and Memory B Size vs. The Number\n of Memories at Capacity Between OCG and WS")
  plt.grid(False)
  plt.legend()
  plt.savefig(f"capacityChart_Overlay.pdf", format='pdf', bbox_inches='tight')
  plt.show()

JOIN_BOTH_Capacity_Simulation(df_OCG, r_min_per_mem_OCG, r_max_per_mem_OCG, n_OCG, brain_region_name_OCG, node_to_idx_dict_OCG, adj_matrix_OCG,
    ws_OCG_k, ws_OCG_n, ws_brain_region_name_OCG, ws_adj_matrix_OCG)



#### Make a WS Graph with tested interference and stability ranges. Then test to verify the WS Graph meets small world metrics.




*   Watts-Strogatz is much more uniform due to the way the nodes are connected to each other than the real world brain connections of OCG. It is possible that OCG has more threshold for misfires, as the graph is not completely connected. Thus, we would not need as many edges to achieve the same connectivity. This is why we have a smaller **d**.
*   n is 400 vs. 409 of OCG. This is a difference of ~10 nodes. Not a large consideration.


MemA and MemB Union size of [200, 210] ==> 30 trials
Use error bars with mean and st dev.

In [ ]:
# Compute Adj_Matrix, Small-World Metrics for WS Graph using OCG Small-World Metrics
import math
ws_OCG_n = 409
ws_OCG_d = 12
ws_OCG_k = int(k_s_OCG)
print("k: ", ws_OCG_k)

ws_OCG_directed = True
ws_OCG_p = compute_p(ws_OCG_n, ws_OCG_d, ws_OCG_directed)
print("p: ", ws_OCG_p)

ws_OCG_knn = math.ceil(ws_OCG_d / 2)
ws_adj_matrix_OCG = create_ws_graph_sparse(ws_OCG_n, ws_OCG_p, ws_OCG_knn)

# Compute WS Small World Metrics
deg_dist_WS_OCG_IN, deg_dist_WS_OCG_OUT = calculate_degree_distribution(ws_adj_matrix_OCG, ws_OCG_directed)
print("In-Degree Distribution: ", np.mean(deg_dist_WS_OCG_IN))
print("Out-Degree Distribution: ", np.mean(deg_dist_WS_OCG_OUT))

# Compute Clustering Coefficient
clustCoeff_WS_OCG = calculate_global_clustering_sparse(ws_adj_matrix_OCG, ws_OCG_directed)
print("Clustering Coefficient: ", clustCoeff_WS_OCG)

# Compute Characteristic Path Length
characteristic_path_length_WS_OCG = calculate_path_length_sparse(ws_adj_matrix_OCG, ws_OCG_directed)
print("Path Length: ", characteristic_path_length_WS_OCG)

# Compute Average Degree of Graph
graph_average_degree_WS_OCG = compute_graph_average_degree(ws_adj_matrix_OCG, ws_OCG_directed)
print("Average Degree of Graph:", graph_average_degree_WS_OCG)


In [ ]:
# Small World Test on WS Graph with Above n,d,k

# Find Graph Average Degree to use in random Gnp graph calculation
print(compute_graph_average_degree(ws_adj_matrix_OCG, ws_OCG_directed))

In [ ]:
# import C_rand, L_rand values from randGraph.ipynb
C_rand_WS = 0.015730598927988813
L_rand_WS = 3.589833550913838
S_WS = compute_small_world_threshold(clustCoeff_WS_OCG, characteristic_path_length_WS_OCG, C_rand, L_rand)
print(S_WS)

if S_WS > 1:
  print("Small world found!")
else:
  print("Not a small world...")

In [ ]:
# Fire Memory A and Memory B in the WS Small Worlds Graph to simulate JOIN
r_min_per_mem_OCG = 117
r_max_per_mem_OCG = 124

# other params needed
ws_brain_region_name_OCG = "WS of OCG Qualities (n, d, k)"
node_to_idx_dict_OCG = [0] * ws_OCG_n

# Fire memory A and memory B together, with varying sizes of 100-125 nodes each.
JOIN_WS_Capacity_Simulation(ws_OCG_k, r_min_per_mem_OCG, r_max_per_mem_OCG, ws_OCG_n, ws_brain_region_name_OCG, node_to_idx_dict_OCG, ws_adj_matrix_OCG)

## Visualize Brain Region Subgraphs

### OCG

In [ ]:
# OVERALL OCG GRAPH
import networkx as nx
import matplotlib.pyplot as plt

# Take adj_sparse (CSR matrix)
# Convert sparse matrix to a NetworkX Graph
G = nx.from_scipy_sparse_array(adj_matrix_OCG, create_using=nx.DiGraph)
pos = nx.spring_layout(G, seed=0)

# Draw the graph
plt.figure(figsize=(8, 8))
nx.draw(
    G,
    node_size=50,          # smaller nodes
    node_color="black",    # black nodes
    edge_color="gray",     # gray edges
    width=0.5,
    alpha=0.7,
    with_labels=False      # turn on if you want node labels
)

plt.show()

In [ ]:
# ZOOMED IN VIEW ON CENTRAL CLUSTER OF NODES
import networkx as nx
import matplotlib.pyplot as plt

# If you already have adj_sparse from your function (CSR matrix)
# Convert sparse matrix to a NetworkX Graph
G = nx.from_scipy_sparse_array(adj_matrix_OCG, create_using=nx.DiGraph)
pos = nx.spring_layout(G, seed=0)

# Draw the graph
plt.figure(figsize=(12, 12))
nx.draw(
    G,
    node_size=50,          # smaller nodes
    node_color="black",    # black nodes
    edge_color="gray",     # gray edges
    width=0.5,
    alpha=0.7,
    with_labels=False      # turn on if you want node labels
)

# ZOOM IN ON THE LARGE CENTRAL CLUSTER
plt.xlim(-0.1, 0.15)   # tighter zoom horizontally
plt.ylim(-0.1, 0.1)   # tighter zoom vertically

plt.show()



In [ ]:
# ZOOMED IN VIEW ON CENTRAL CLUSTER OF NODES
import networkx as nx
import matplotlib.pyplot as plt

# If you already have adj_sparse from your function (CSR matrix)
# Convert sparse matrix to a NetworkX Graph
G = nx.from_scipy_sparse_array(adj_matrix_OCG, create_using=nx.DiGraph)
pos = nx.spring_layout(G, seed=0)

# Draw the graph
plt.figure(figsize=(12, 12))
nx.draw(
    G,
    node_size=50,          # smaller nodes
    node_color="black",    # black nodes
    edge_color="gray",     # gray edges
    width=0.5,
    alpha=0.7,
    with_labels=True,
    font_size=15,
    font_color="red"
)

# # Add labels separately with an offset (d_y shift upwards)
# labels = {n: str(n) for n in G.nodes()}
# label_pos = {n: (x, y + 0.00000001) for n, (x, y) in pos.items()}

# nx.draw_networkx_labels(
#     G,
#     label_pos,
#     labels,
#     font_size=8,
#     font_color="red"   # choose a contrasting color
# )

# ZOOM IN ON THE LARGE CENTRAL CLUSTER
plt.xlim(-0.1, 0.15)   # tighter zoom horizontally
plt.ylim(-0.1, 0.1)   # tighter zoom vertically

plt.show()



In [ ]:
# OVERALL LH_L GRAPH
import networkx as nx
import matplotlib.pyplot as plt

# Take adj_sparse (CSR matrix)
# Convert sparse matrix to a NetworkX Graph
G = nx.from_scipy_sparse_array(adj_matrix, create_using=nx.DiGraph)
pos = nx.spring_layout(G, seed=0)

# Draw the graph
plt.figure(figsize=(8, 8))
nx.draw(
    G,
    node_size=50,          # smaller nodes
    node_color="black",    # black nodes
    edge_color="gray",     # gray edges
    width=0.5,
    alpha=0.7,
    with_labels=False      # turn on if you want node labels
)

plt.show()

In [ ]:
# ZOOMED IN VIEW ON CENTRAL CLUSTER OF NODES

# Convert sparse matrix to a NetworkX Graph
G = nx.from_scipy_sparse_array(adj_matrix, create_using=nx.DiGraph)
pos = nx.spring_layout(G, seed=0)

# Draw the graph
plt.figure(figsize=(12, 12))
nx.draw(
    G,
    node_size=50,          # smaller nodes
    node_color="black",    # black nodes
    edge_color="gray",     # gray edges
    width=0.5,
    alpha=0.7,
    with_labels=False      # turn on if you want node labels
)

# ZOOM IN ON THE LARGE CENTRAL CLUSTER
plt.xlim(-0.35, 0.3)   # tighter zoom horizontally
plt.ylim(-0.3, 0.35)   # tighter zoom vertically

plt.show()
